# Bitmask to polygonal annotation
This notebook contains functions that allow us to transform bitmasks to polygonal annotations in COCO format
## Main code

In [4]:
from PIL import Image # (pip install Pillow)
import matplotlib.pyplot as plt
import numpy as np                                 # (pip install numpy)
from skimage import measure                        # (pip install scikit-image)
from shapely.geometry import Polygon, MultiPolygon # (pip install Shapely)
import cv2
import os

In [5]:

def create_submask(my_img):
    sub_masks = {}
    width, height, _ = my_img.shape
    black = np.ones(3)*255

    for x in range(width):
        for y in range(height):
            
            color = my_img[x][y][:3]
            if((black==color).all()):
                pixel_str = str(color)
                sub_mask = sub_masks.get(pixel_str)
                if sub_mask is None:
                # Create a sub-mask (one bit per pixel) and add to the dictionary
                    # Note: we add 1 pixel of padding in each direction
                    # because the contours module doesn't handle cases
                    # where pixels bleed to the edge of the image
                    sub_masks[pixel_str] = np.zeros((width+2, height+2)) 
                # Set the pixel value to 1 (default is 0), accounting for padding
                sub_masks[pixel_str][x+1][y+1]=255
    return sub_masks



In [6]:
def create_submask_annotation(sub_mask, image_id, category_id, annotation_id, is_crowd):
    # Find contours (boundary lines) around each sub-mask
    # Note: there could be multiple contours if the object
    # is partially occluded. (E.g. an elephant behind a tree)
    contours = measure.find_contours(sub_mask, 0.5, positive_orientation='low')

    segmentations = []
    polygons = []
    for contour in contours:
        # Flip from (row, col) representation to (x, y)
        # and subtract the padding pixel
        for i in range(len(contour)):
            row, col = contour[i]
            contour[i] = (col - 1, row - 1)

        # Make a polygon and simplify it
        poly = Polygon(contour)
        poly = poly.simplify(1.0, preserve_topology=False)
        polygons.append(poly)
        segmentation = np.array(poly.exterior.coords).ravel().tolist()
        segmentations.append(segmentation)

    # Combine the polygons to calculate the bounding box and area
    multi_poly = MultiPolygon(polygons)
    x, y, max_x, max_y = multi_poly.bounds
    width = max_x - x
    height = max_y - y
    bbox = (x, y, width, height)
    area = multi_poly.area

    annotation = {
        'segmentation': segmentations,
        'iscrowd': is_crowd,
        'image_id': image_id,
        'category_id': category_id,
        'id': annotation_id,
        'bbox': bbox,
        'area': area
    }

    return annotation

In [12]:
def images_to_annotations(dir_path):
    #Initializing images and annotations lists
    images=[]
    annotations = []
    img_names = os.listdir(dir_path)
    img_license = 0
    img_id=0
    ann_id=0
    is_crowd=0
    print (img_names)        
    for img_name in img_names:
        img_path = os.path.join(dir_path, img_name)
        img = cv2.imread(img_path)
        img_size = img.shape
        submasks = create_submask(img)
        for color, submask in submasks.items():
            img_cat = color2cat[color]
            annotation = create_submask_annotation(submask , img_id, img_cat, ann_id, is_crowd)
            annotations.append(annotation)
            ann_id+=1
            

        new_img={}
        new_img["license"] = img_license
        new_img["file_name"] = img_name.split(".")[0]+".jpg" #Changed to match the video images
        new_img["width"] = img_size[1]
        new_img["height"] = img_size[0]
        new_img["id"] = img_id
        images.append(new_img)

        # sub_masks = create_sub_masks()
        # for color, sub_mask in sub_masks.items():
        #     plt.imshow(sub_mask)
        #     plt.show()
        
        img_id+=1

    return annotations,images





In [8]:
my_dict = {}
color2cat={"[255 255 255]": 1}
info = {
      "description":"Test Dataset",
      "url":"",
      "version":"0.1",
      "year":2020,
      "contributor":"Josmar Suarez",
      "date_created":"2020/07/14"
   }

licenses = [{
        "url": "",
        "id": 0,
        "name": "Attribution-NonCommercial-ShareAlike License"
    }]
categories = [
      {
         "supercategory":"person",
         "id":1,
         "name":"person"
      }
   ]

In [13]:
dir_path = "/home/josmar/proyectos/codes/annotation_tools/background_substraction/bin_close_images/input"
anns,imgs = images_to_annotations(dir_path)

my_dict["info"]= info
my_dict["licenses"]= licenses
my_dict["images"]=imgs
my_dict["categories"]=categories
my_dict["annotations"]=anns
print(my_dict.keys())

['0218_img.png', '0172_img.png', '0254_img.png', '0017_img.png', '0122_img.png', '0139_img.png', '0054_img.png', '0262_img.png', '0057_img.png', '0002_img.png', '0259_img.png', '0126_img.png', '0148_img.png', '0106_img.png', '0136_img.png', '0185_img.png', '0081_img.png', '0086_img.png', '0236_img.png', '0135_img.png', '0176_img.png', '0190_img.png', '0242_img.png', '0032_img.png', '0013_img.png', '0147_img.png', '0166_img.png', '0138_img.png', '0205_img.png', '0077_img.png', '0155_img.png', '0098_img.png', '0150_img.png', '0223_img.png', '0191_img.png', '0113_img.png', '0053_img.png', '0055_img.png', '0283_img.png', '0060_img.png', '0267_img.png', '0073_img.png', '0274_img.png', '0160_img.png', '0241_img.png', '0294_img.png', '0187_img.png', '0107_img.png', '0016_img.png', '0028_img.png', '0276_img.png', '0074_img.png', '0235_img.png', '0022_img.png', '0203_img.png', '0284_img.png', '0084_img.png', '0215_img.png', '0093_img.png', '0120_img.png', '0085_img.png', '0237_img.png', '0268_i

In [15]:
import json

with open('casia.json', 'w') as fp:
    json.dump(my_dict, fp)

## trying other methods

### Could be found at:
https://github.com/cocodataset/cocoapi/issues/131

In [19]:
import json
import numpy as np
from pycocotools import mask
from skimage import measure


ground_truth_binary_mask = cv2.imread('/home/josmar/proyectos/codes/annotation_tools/background_substraction/bin_close_images/input/0001_img.png',0)
fortran_ground_truth_binary_mask = np.asfortranarray(ground_truth_binary_mask)
encoded_ground_truth = mask.encode(fortran_ground_truth_binary_mask)
ground_truth_area = mask.area(encoded_ground_truth)
ground_truth_bounding_box = mask.toBbox(encoded_ground_truth)
contours = measure.find_contours(ground_truth_binary_mask, 0.5)

annotation = {
        "segmentation": [],
        "area": ground_truth_area.tolist(),
        "iscrowd": 0,
        "image_id": 123,
        "bbox": ground_truth_bounding_box.tolist(),
        "category_id": 1,
        "id": 1
    }

for contour in contours:
    contour = np.flip(contour, axis=1)
    segmentation = contour.ravel().tolist()
    annotation["segmentation"].append(segmentation)
    
print(json.dumps(annotation, indent=4))

6666,
            1225.8333333333333,
            566.0,
            1225.0,
            566.8333333333334
        ],
        [
            1268.0,
            566.9166666666666,
            1267.0833333333333,
            566.0,
            1268.0,
            565.0833333333334,
            1268.9166666666667,
            566.0,
            1268.0,
            566.9166666666666
        ],
        [
            1057.0,
            567.9285714285714,
            1056.0714285714287,
            567.0,
            1057.0,
            566.0714285714286,
            1057.9285714285713,
            567.0,
            1057.0,
            567.9285714285714
        ],
        [
            1266.0,
            575.9166666666666,
            1265.0,
            575.9,
            1264.0,
            575.75,
            1263.25,
            575.0,
            1264.0,
            574.25,
            1265.0,
            574.1,
            1265.1666666666667,
            574.0,
            1265.16666